## Using BlockEdge API to do some basic waves analytics
### https://www.blockedge.app
#### BlockEdge is a beta version search engine devised by MOBR Systems. With BlockEdge users can search their keywords on waves blockchain content

#### Functions get and search_query will be the endpoints to use BlockEdge API

In [1]:
import requests, json

BE_URL = "https://www.blockedge.app"
timeout = 60 * 1000
headers = {'accept': 'application/json', 'content-type': 'application/json'}

def get(url:str) -> dict:
    res = requests.get(url=url, timeout=timeout, headers=headers)
    if res.status_code == 200:
        rj = json.loads(res.json())
        res.close()
        if rj.get('error'):
            print(rj)
            return dict()

        return rj

    res.close()
    return dict()

def search_query(query:str, page:int=1) -> dict:
    return get(f'{BE_URL}/search/waves/{query}/{page}')

#### Let's consider a developer devising a poker dapp. To see if there is movement around the poker worlds in the waves blockchain, an intersting thing to do would be search for the "poker" keyword in the blocks' content:

In [2]:
query = 'poker'
res = search_query(query)
if res:
    print (f"number of ocurrences for '{query}': {res['ocurrences']} in {res['blocks']} blocks")
else:
    print ('no records found')

number of ocurrences for 'poker': 254 in 64 blocks


#### Now let's consider an investor wanting to know how many transactions were realized on waves

In [3]:
query = 'count:transactions'
res = search_query(query)
if res:
    print (f"number of transactions on waves: {res['results']['count_res']}")
else:
    print ('something wrong')

number of transactions on waves: {'transactions': 466347}


#### However, a more interesting approach would be how is the number of transactions over time

In [4]:
import pandas as pd

query = 'get:transactions,timestamp'
res = search_query(query)
if res:
    table = res['results']['get_res']['recs']
    headers = res['results']['get_res']['attrs']
    df = pd.DataFrame(table, columns=headers)
    print(df.head(10))
else:
    df = pd.DataFrame()
    print ('something wrong')

   transactionCount      timestamp
0                16  1669734649585
1                17  1669379455948
2                16  1669248313499
3                12  1669248313499
4                52  1669248210603
5                 6  1669094316328
6                59  1669067399206
7               108  1669067154810
8                 9  1669066942642
9                11  1669066735447


#### And visualizing all returned records, but aggregating transactions by week

In [5]:
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objects as go

if not df.empty:
    agg_dict = {
        'timestamp': 'first',
        'transactionCount': 'sum'
    }
    df['timestamp'] = pd.to_timedelta(df['timestamp'], 'ms')
    df = df.set_index(df['timestamp']).resample('7d', label='left', closed='left').agg(agg_dict)
    df['date'] = pd.to_datetime(df['timestamp'], unit='ns').dt.strftime('%Y-%m-%d')

    init_notebook_mode(connected=True)
    data = [go.Bar(x=df.date, y=df.transactionCount)]
    iplot(data, filename='jupyter-basic_bar')

## Conclusion

#### Building an accurate overview of the current and historical state of blockchains is a hard task. Generally, there is a difficulty of understanding and accessing blockchain metrics. Currently, users without programming skills depend on blockchain explorers to have access to these metrics, but with a limited and static experience. Even with a search engine like BlockEdge with some syntatic sugar to research blockchain metrics, users still need programming skills to build their overviews. It is key to the blockchain community a system to ease how users specify their queries and to assist users to create, visualize, and share blockchain analytics in the form of informative artifacts.